In [1]:
# you will be prompted with a window asking to grant permissions
# click connect to google drive, choose your account, and click allow
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [2]:
# TODO: fill in the path in your Google Drive in the string below
# Note: do not escape slashes or spaces in the path string
import os
os.chdir("/content/drive/MyDrive/cs444-project_falavise") # fill in with the path to the google drive folder where your code is.

In [3]:
# reload imports when running a function(this will make it so you don't have to reload the entire notebook if you change a function that you are importing and want the change to register)
%reload_ext autoreload
%autoreload 2

In [4]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.8/252.8 kB 34.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.6 MB/s eta 0:00:00


In [7]:
os.mkdir('data')
os.chdir('data')

In [8]:
!wget https://saurabhg.web.illinois.edu/teaching/cs444/fa2023/mp3/coco.zip -O coco.zip
!unzip coco.zip

Streaming output truncated to the last 5000 lines.
  inflating: coco/train2017/000000252846.jpg  
  inflating: coco/train2017/000000196664.jpg  
  inflating: coco/train2017/000000057992.jpg  
  inflating: coco/train2017/000000028820.jpg  
  inflating: coco/train2017/000000377422.jpg  
  inflating: coco/train2017/000000099543.jpg  
  inflating: coco/train2017/000000467318.jpg  
  inflating: coco/train2017/000000178971.jpg  
  inflating: coco/train2017/000000477679.jpg  
  inflating: coco/train2017/000000564611.jpg  
  inflating: coco/train2017/000000574402.jpg  
  inflating: coco/train2017/000000160858.jpg  
  inflating: coco/train2017/000000073830.jpg  
  inflating: coco/train2017/000000098107.jpg  
  inflating: coco/train2017/000000052168.jpg  
  inflating: coco/train2017/000000568765.jpg  
  inflating: coco/train2017/000000561027.jpg  
  inflating: coco/train2017/000000391825.jpg  
  inflating: coco/train2017/000000276066.jpg  
  inflating: coco/train2017/000000225358.jpg  
  inflati

In [ ]:
#### Preprocess Cococ data

In [22]:
import pickle


from pycocotools.coco import COCO
from torchtext.vocab import GloVe, FastText
import torchtext.vocab as vocab

In [10]:
def compute_cat_mappings(ann_file: str, glove) \
        -> dict[str, dict]:
    """
    Compute mappings from COCO categories to GloVe/fasttext indices.

    Args:
        ann_file: Path to the annotation file.
        glove: embeddingss.

    Returns:
        dict: A dictionary containing the mappings. The keys are:
            - "stoi" (dict): COCO category to emb index.
            - "itos" (dict): emd index to COCO category.
    """
    print("Computing category mappings...", end=" ")
    coco = COCO(ann_file)
    cat_mappings = {"stoi": {}, "itos": {}}
    for cat in coco.cats.values():
        for token in cat["name"].split(" "):
            cat_mappings["stoi"][token] = glove.stoi[token]
            cat_mappings["itos"][glove.stoi[token]] = token
    print("Done.")
    return cat_mappings

In [11]:
os.chdir('../')

In [30]:
root = "./data/coco"
glove = GloVe(name="6B", dim=300)
model_path = 'wiki-news-300d-1M.vec'

# Create a custom Vectors object
url = 'https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip'
fasttext = vocab.Vectors(name='wiki-news-300d-1M.vec', url=url)

# Compute the category mappings.
ann_file = os.path.join(root, "annotations/instances_train2017-animal-nocrowd.json")
#use either glove or fasttext depending on your choice in compute_cat_mappings
cat_mappings = compute_cat_mappings(ann_file, fasttext)
with open(os.path.join(root, "cat_mappings.pkl"), "wb") as f:
    pickle.dump(cat_mappings, f)

100%|██████████| 999994/999994 [02:15<00:00, 7388.44it/s]


Computing category mappings... loading annotations into memory...
Done (t=0.26s)
creating index...
index created!
Done.


In [33]:
!python train_explainer.py --model resnet18 --layer-target layer4 --batch-size 16 --name resnet18_fasttext_25 --word-model fasttext --layer-classifier fc --refer coco --epochs 25

args=Namespace(batch_size=16, cat_frac=0.7, dir_data='./data', dir_save='./outputs', epochs=25, layer_classifier='fc', layer_target='layer4', margin=1.0, model='resnet18', name='resnet18_fasttext_25', num_workers=8, path_model=None, random=False, refer='coco', word_model='fasttext', save_every=1000, seed=42, train_frac=0.9, wandb=False, word_embedding_dim=300)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100% 44.7M/44.7M [00:00<00:00, 105MB/s]
No model found at './outputs/resnet18_fasttext_25/ckpt_best.pth.tar'. Training from scratch.
loading annotations into memory...
Done (t=0.48s)
creating index...
index created!
loading annotations into memory...
Done (t=0.10s)
creating index...
index created!
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, 

In [35]:
!python infer_filter.py --model resnet18 --layer-target layer4 --layer-classifier fc --refer coco --name resnet18_fasttext_25 --path-model "outputs/resnet18_fasttext_25/ckpt_best.pth.tar" --word-model fasttext --s 20

args=Namespace(batch_size=1, dir_data='./data', dir_save='./outputs', layer_classifier='fc', layer_target='layer4', method='projection', model='resnet18', name='resnet18_fasttext_25', num_heatmaps=5, num_tokens=10, num_workers=8, p=25, path_model='outputs/resnet18_fasttext_25/ckpt_best.pth.tar', random=False, refer='coco', s=20, word_model='fasttext', seed=42, threshold_iou=0.04, u=[0, 12, 24, 36, 48, 60, 72, 84, 96, 108, 120, 132, 144, 156, 168, 180, 192, 204, 216, 228, 240, 252], wandb=False, weigh_s_by_relevance=False, word_embedding_dim=300)
Loading model from 'outputs/resnet18_fasttext_25/ckpt_best.pth.tar'... Done.
loading annotations into memory...
Done (t=0.59s)
creating index...
index created!
loading annotations into memory...
Done (t=0.09s)
creating index...
index created!
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, 